# Introduction

## Using Jupyter

Jupyter is a front end to the [IPython](https://ipython.org) interactive shell, and offers IDE like features.  It is separated into two main types of cell: [Markdown](https://en.wikipedia.org/wiki/Markdown) cells (such as this one) which allows markdown or HTML code to be written, and code cells like the next one which can be run in real time.

To execute code in a cell, press `Crtl` + `Enter`, click on the `[ > ]Run` button in the main menu, or press `Shift` + `Enter` if you wish to execute the code and then move on to a new cell (creating it if it does not already exist).

## Setup

MongoDB is a NoSQL database, which has a core API in JavaScript, and a series of other APIs in different languages.  The one we are going to use is the Python API, [PyMongo](https://api.mongodb.com/python/current/).  MongoDB instance on your VM is already started by default.

PyMongo is a package that contains tools to work with MongoDB from Python. We have installed it in the VM provided to you. 

This imports the `MongoClient` class from the pymongo module, which we will use to deal with all our connections from.  We're connecting to our localhost, which is listening on port 27017. There are more options, the documentation for the formatting of the connection string is at https://docs.mongodb.com/manual/reference/connection-string/.

In [2]:
!pip install pymongo

    100% |████████████████████████████████| 409kB 1.5MB/s ta 0:00:01
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')

If you do not get any errors, you have confirmed PyMongo library has been successfully installed&configured in the VM. We will now check to see whether the connection is correct.  The following code calls a function which returns a list of all current databases.  If your Mongo instance is still empty, it should be something like `['admin', 'local']`.

In [19]:
client.list_database_names()

['admin',
 'client-app-1',
 'client-app-2',
 'conFusion',
 'local',
 'mongo_lab',
 'mycustomers']

Next, we are going to create a database object `db`, which is a property of the `client` object.  MongoDB is schemaless, and so accessing a database like this will create the database if it does not already exist.

A database can be accessed by using "dot" notation (i.e., `client.dbname`), or dictionary notation (i.e., `client['dbname']`).  This also applies to making collections

Create a database called `test` in a variable called `db`.  Using that variable, create a collection called `test_collection` with a variable called `collection` as follows.  Run the code in the following cell (there should not be any output)

In [14]:
# Create database and collection objects for convenience
db = client.mongo_lab
collection = db.test_collection

# Using MongoDB

## Inserting data

MongoDB data is stored as BSON (binary JSON), which is essentially JSON with some additional optimisations, so the way to insert data is as a JSON object.  For Python, you can use a `dict` or a `list` for this, and then call either `insert_one` or `insert_many` on the collection.  

In [18]:
# Create an object and insert into the `test_collection`
single_obj = {'name': 'Amber', 'star_sign': 'Capricorn', 'favourite_song': 'The Load-Out'}
collection.insert_one(single_obj)
single_obj_2 = {'name': 'Huw', 'star_sign': 'Libra', 'favourite_song': 'The masses against the classes'}
collection.insert_one(single_obj_2)
single_obj_3 = {'name': 'Robert', 'star_sign': 'Leo', 'favourite_song': 'Bad day'}
collection.insert_one(single_obj_3)

We will look at querying data in more detail below, but for now, to see whether the object got successfully inserted into the collection, run the code below.  This will always return the first instance which matches the query.  You will notice that even though we didn't specify `_id` one got added already.  This is a unique identifier for the document in the collection

In [20]:
collection.find_one() # returns a single document matching the query condition

{'_id': ObjectId('5c09b5f9651cff367fe07129'),
 'favourite_song': 'The Load-Out',
 'name': 'Amber',
 'star_sign': 'Capricorn'}

In [21]:
collection.find_one({'name':'Robert'})

{'_id': ObjectId('5c09b5f9651cff367fe0712b'),
 'favourite_song': 'Bad day',
 'name': 'Robert',
 'star_sign': 'Leo'}

Remember, for MongoDB, you do not have to specify a schema or create a collection, it will be created automatically.  You don't need to keep to the same layout, but can have entirely different objects.  Consider the following: 

In [22]:
from datetime import datetime
obj1 = {'Meaning of life': 42}
obj2 = {'ABC': 'DEF', 'time': datetime.now()}
collection.insert_one(obj1)
collection.insert_one(obj2)

We can also use the `insert_many`, which accepts a list of dicts.  In the cell below, create a list of dicts called `many_objects`, and call the `insert_many` function.  The code below that will iterate over all the documents in the database.

In [26]:
from pprint import pprint

In [27]:
# YOUR CODE HERE

list_of_dicts = [
    {
        "name": "Mini",
        "age": 23,
        "planet": "unknown"
    },
    {
        "name": "Paddy",
        "age": 25
    },
    {
        "name": "Saurav",
        "planet": "Earth"
    }
]

# collection.insert_many(list_of_dicts)


#See what has been inserted into the collection
for doc in collection.find():
    pprint(doc)

{'_id': ObjectId('5c09b5f9651cff367fe07129'),
 'favourite_song': 'The Load-Out',
 'name': 'Amber',
 'star_sign': 'Capricorn'}
{'_id': ObjectId('5c09b5f9651cff367fe0712a'),
 'favourite_song': 'The masses against the classes',
 'name': 'Huw',
 'star_sign': 'Libra'}
{'_id': ObjectId('5c09b5f9651cff367fe0712b'),
 'favourite_song': 'Bad day',
 'name': 'Robert',
 'star_sign': 'Leo'}
{'Meaning of life': 42, '_id': ObjectId('5c09b642651cff367fe0712c')}
{'ABC': 'DEF',
 '_id': ObjectId('5c09b642651cff367fe0712d'),
 'time': datetime.datetime(2018, 12, 6, 23, 52, 34, 776000)}
{'_id': ObjectId('5c09b741651cff367fe0712e'),
 'age': 23,
 'name': 'Mini',
 'planet': 'unknown'}
{'_id': ObjectId('5c09b741651cff367fe0712f'), 'age': 25, 'name': 'Paddy'}
{'_id': ObjectId('5c09b741651cff367fe07130'),
 'name': 'Saurav',
 'planet': 'Earth'}


## Importing and querying data

For this part of the exercise, we will use a sample dataset provided by Mongo  for a documentation tutorial.  The following cell runs the `mongoimport` command, which is a Unix command which comes with Mongo for importing data. We will need to run a bash command in the next cell first.  This uses the Jupyter \"magics\", and requires that the first line include `%%bash`. The code does the following:

- Download the JSON file from the url, and save as ./primer-dataset.json
- Import into the `test` database into the collection `restaurants` whilst dropping any collection which already exists from the file ./primer-dataset.json
- Deletes the file.

 Click on the following cell, and execute it:

In [28]:
%%bash
# Use wget to download the data
wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
# mongoimport is the Mongo command to import data.  
# It specifies the database, collection and format, and import file
# --drop means it's going to drop any collection with the same name which already exists
mongoimport --db mongo_lab --collection restaurants --drop --file ./primer-dataset.json
# Delete the JSON file we just downloaded
rm ./primer-dataset.json

--2018-12-06 23:57:58--  https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.16.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.16.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11874761 (11M) [text/plain]
Saving to: ‘primer-dataset.json’

     0K .......... .......... .......... .......... ..........  0% 1.78M 6s
    50K .......... .......... .......... .......... ..........  0% 3.40M 5s
   100K .......... .......... .......... .......... ..........  1% 7.36M 4s
   150K .......... .......... .......... .......... ..........  1% 2.46M 4s
   200K .......... .......... .......... .......... ..........  2% 3.91M 4s
   250K .......... .......... .......... .......... ..........  2% 1.87M 4s
   300K .......... .......... .......... .......... ..........  3% 4.70M 4s
   350K .......... .......... .......... .......... .

Change the variable `collection` to refer to the new collection `restaurants`, and inspect the general format of the data by adding the code below to find the first record of the collection:

In [29]:
# YOUR CODE HERE

collection = db.restaurants

We saw the [`collection.find()`](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find) function earlier to return all the documents we inserted into our `test` collection.  Without any arguments, `find()` will return a cursor of all the available documents from in the collection.  To refine queries, however, the search can be filtered by the addition of a first parameter.

The `filter` parameter is a dict, which searches for the documents where `key` = `value` where the dict is of the form `{key: value}`.  For example, to find all bakeries in the city, we would do the following query:   

**WARNING** Unlike the Mongo command line interface, if you try and print the output of a `find()` query, it will continue to output all results until it has finished.  This can cause the browser to crash, particularly if it is a particularly large query set.

Using `find().count()` is a useful way of checking how many a result will return, and `find_one()` to see the general structure of a result. If you use `find()`, make sure you either include the `limit` argument, or have a counter or other condition to break out of your printing loop!

In [31]:
for doc in collection.find({'cuisine': 'Bakery'}).limit(5):
    pprint(doc)

{'_id': ObjectId('5c09b78cd8c8b5898b20c861'),
 'address': {'building': '1007',
             'coord': [-73.856077, 40.848447],
             'street': 'Morris Park Ave',
             'zipcode': '10462'},
 'borough': 'Bronx',
 'cuisine': 'Bakery',
 'grades': [{'date': datetime.datetime(2014, 3, 3, 0, 0),
             'grade': 'A',
             'score': 2},
            {'date': datetime.datetime(2013, 9, 11, 0, 0),
             'grade': 'A',
             'score': 6},
            {'date': datetime.datetime(2013, 1, 24, 0, 0),
             'grade': 'A',
             'score': 10},
            {'date': datetime.datetime(2011, 11, 23, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2011, 3, 10, 0, 0),
             'grade': 'B',
             'score': 14}],
 'name': 'Morris Park Bake Shop',
 'restaurant_id': '30075445'}
{'_id': ObjectId('5c09b78cd8c8b5898b20c879'),
 'address': {'building': '120',
             'coord': [-73.9998042, 40.7251256],
 

Noted, count() is deprecated in the MongoDb drivers compatible with the 4.0features, as a result, we should use countDocuments() in the following exercises. (for more information, check https://docs.mongodb.com/manual/reference/method/db.collection.count/ and http://api.mongodb.com/python/current/changelog.html)

In [32]:
collection.count_documents({'cuisine': 'Bakery'})

691

A filter can have as many conditions as you like, and will assume that you are using an AND condition, unless you specify otherwise (as below).  In the cell below, write a query to return the number/count of all the establishments with a cuisine of `Hamburgers` in the borough of Manhattan.

In [33]:
# YOUR CODE HERE
# All establishments with: 
# * a cuisine of 'Hamburgers' 
# * in the borough of 'Manhattan

collection.count_documents({'cuisine': 'Hamburgers', 'borough':'Manhattan'})



124

In [34]:
collection.distinct('cuisine')

['Hamburgers',
 'Irish',
 'Jewish/Kosher',
 'American',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Bakery',
 'Chicken',
 'Caribbean',
 'Turkish',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Pizza',
 'Continental',
 'Italian',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'Polish',
 'German',
 'Steak',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'African',
 'Not Listed/Not Applicable',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japanese',
 'Filipino',
 'Juice, Smoothies, Fruit Salads',
 'Brazilian',
 'Afghan'

### Sub-documents

A valid JSON style "document" can have another JSON document inside it.  To access these, we use the "dot" notation to access them.  For example, to get all the restaurants in a certain zipcode, you would run code as follows:

In [38]:
from pprint import pprint
cursor = collection.find({'address.zipcode': '10462'}, {'address.zipcode': 1, '_id':0}).limit(5)
for c in cursor:
    pprint(c)

{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}
{'address': {'zipcode': '10462'}}


### Operators

MongoDB has a series of [operators](https://docs.mongodb.com/manual/reference/operator/query/) which allow us to do more sophisticated filters on our queries.  There are too many to go into individually, but we will look at a few important ones.  The specific syntax varies depending on the operator, so it isn't possible to give a general rule, but we will go over a few examples here.  Make sure you check the [documentation](https://docs.mongodb.com/manual/reference/operator/query/) for use on each one.

#### [\$or](https://docs.mongodb.com/manual/reference/operator/query/or/#op._S_or)

Performs a logical **OR** operation on all the key/value pairs in a list, as in the code below:

In [41]:
filter = {"$or": [{"cuisine": "Polynesian"}, {"cuisine": "Hawaiian"}]}
for f in collection.find(filter, {'name': 1, '_id': 0}).limit(2):
    pprint(f)

{'name': 'Camaradas El Barrio'}
{'name': 'Makana'}


#### [`$regex`](https://docs.mongodb.com/manual/reference/operator/query/regex/#op._S_regex)

The `$regex` operator searches for a regular expression on a particular field.  Within the filter field, the named field (a key) takes a dict as a value.  

For example, to search for all restaurants which start with the word "Pretzel" in the title you can do the following:

In [42]:
filter = {"name": {"$regex": '^Pretzel'}}
collection.count_documents(filter)

4

There are other ways to use regular expressions in PyMongo, you can use the [`re`](https://docs.python.org/3/library/re.html) module in Python.  In Mongo itself, you can use the following syntax: The simplest is to enclose the regular expression inside `/` characters, as in `{"name": /^Pretzel/}`, but that doesn't work properly in PyMongo.

Using the `$regex` operator, find all restaurants which end in "Bar" in the borough of Brooklyn.

HINT: The regex character for the end of a string is `$`

In [44]:
# YOUR CODE HERE
filter_regex = {"name": {"$regex": "Bar$"}}

collection.count_documents(filter_regex)

653

#### [`$gt`](https://docs.mongodb.com/manual/reference/operator/query/gt/#op._S_gt)

The `$gt` operator is a comparison between two values where one is greater than the other.  

For example, consider this code which finds restaurants which have had a score of more than 15:

In [ ]:
filter = {'grades.score': {'$gt': 12}}
collection.count_documents(filter)

Using one of the other comparison operators, find all restaurants which had a grade awarded on the 15 December 2012.  You'll need to create a [`datetime`](https://docs.python.org/2/library/datetime.html#datetime-objects) object in Python.

In [53]:
# YOUR CODE HERE

import datetime

date_to_filter = datetime.datetime(2012, 12, 15)

for doc in collection.find({'grades.date': date_to_filter}, {'_id': 0, 'grades': 1, 'name': 1}):
    pprint (doc)

{'grades': [{'date': datetime.datetime(2014, 8, 5, 0, 0),
             'grade': 'A',
             'score': 10},
            {'date': datetime.datetime(2014, 2, 5, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2012, 12, 15, 0, 0),
             'grade': 'A',
             'score': 12},
            {'date': datetime.datetime(2011, 11, 23, 0, 0),
             'grade': 'A',
             'score': 12}],
 'name': 'Ajanta India'}
{'grades': [{'date': datetime.datetime(2014, 12, 16, 0, 0),
             'grade': 'A',
             'score': 4},
            {'date': datetime.datetime(2013, 12, 23, 0, 0),
             'grade': 'A',
             'score': 9},
            {'date': datetime.datetime(2012, 12, 15, 0, 0),
             'grade': 'A',
             'score': 10}],
 'name': 'Internationalhouse'}
{'grades': [{'date': datetime.datetime(2015, 1, 14, 0, 0),
             'grade': 'A',
             'score': 6},
            {'date': datetime.datetime

## Organising output

So far, we have seen the two of the arguments in the `find()` and related functions.  The `filter` which allows us to select the criteria for documents in the collection, and the `limit` to limit the amount of results.  You should read the documentation fully about the function in your own time, but for now, we will go over two other arguments which are for organising output: field selection, and sorting.

The field selection or `projection` argument is the argument after the \[optional\] filter, and is either:

* A list of fields to include (plus \_id)
* A dict of fields with True/False to include

For example, to display only the name of the restaurant:

In [54]:
filter = {'cuisine': 'Brazilian'}
fields = {'_id': False, 'name': True}
collection.find_one(filter, fields)

{'name': 'Churrascaria Plataforma'}

The sort argument is a dict object of field names as keys, and directions.  This can be done either as a named parameter when calling `find()`, or as a function in its own right [`sort()`](https://api.mongodb.com/python/current/api/pymongo/cursor.html#pymongo.cursor.Cursor.sort)

For example, to sort in alphabetical order, consider the following code:

In [55]:
import pymongo
# The ASCENDING and DESCENDING constants have values of 1 (ASCENDING) and -1 (DESCENDING)
sort = [('name', pymongo.ASCENDING)]
for d in collection.find(filter, projection=fields, sort=sort):
    pprint(d)

{'name': 'Barzinho'}
{'name': 'Beco'}
{'name': 'Beija-Flor'}
{'name': 'Berimbau'}
{'name': 'Brazil Brazil Restuarant'}
{'name': 'Brazil Grill'}
{'name': 'Buffet 58'}
{'name': 'Carioca Grill'}
{'name': 'Casa'}
{'name': 'Churrascaria Plataforma'}
{'name': 'Circus Restaurant'}
{'name': 'Copacabana Pizza & Grill'}
{'name': 'Emporium Brasil Restaurant'}
{'name': 'Esperanto'}
{'name': 'Fogo De Chao'}
{'name': 'Malagueta  Restaurant'}
{'name': 'Miss Favela'}
{'name': 'Pacificos Fine Foods'}
{'name': 'Point Brazil'}
{'name': 'Rainhas'}
{'name': 'Rice & Beans Restaurant'}
{'name': 'Sao Restaurant'}
{'name': 'Texas De Brazil Churrascaria'}
{'name': 'Via Brazil'}
{'name': 'Villa Brazil Cafe Grill'}
{'name': 'Zebu Grill'}


# MongoDB Aggregation Framework

The most common usage for the aggregation framework is to perform group operations such as sum, count or average.  The framework works as a pipeline, with a series of different stages where the data are transformed in each one.

At its simplest, this can be used to obtain output like min, max, count, avg on a collection as follows:

[link](https://docs.mongodb.com/manual/reference/operator/aggregation/group/)

In [56]:
group = {
    '$group': {
        '_id': None, 
        'size': {'$sum': 1},
        'min': {'$min': '$restaurant_id'},
        'max': {'$max': '$restaurant_id'}
    }
}

cursor = collection.aggregate([group])
for c in cursor:
    print(c)

{'_id': None, 'size': 25359, 'min': '30075445', 'max': '50018995'}


Note that it has an `$_id: None` key/value pair in it.  It is compulsory for a `$group` pipeline to have one, and it indicates what it is grouping by.  In this case, we haven't grouped it at all, however it can also be used for more complex output where documents are grouped according to a field.

### Aggregation example

Consider this example, of finding the breakdown of how many of each type of restaurant there is in the Bronx.  We would need to go through the following stages:

- Identify restaurants which are in the Bronx
- Group the restaurants by type to get the count
- Sort the results in a sensible way

The code to perform this query is below:

In [60]:
# Restrict the results to only establishments in the Bronx.  
# '$match' indicates the stage in the pipeline, and the dictionary is the same as using with find()
match = {
    "$match": {"borough": "Bronx"}
}

# $group indicates the stage in the pipeline
# _id is the field to perform the operation on (like SQL GROUP BY)
# count is the name of the field that the result will be in
# $sum is the counting operation, and the value 1 is how many to count each time
group = {
    '$group': {'_id': '$cuisine', 'count': {'$sum': 1}}
    
}
# $sort indicates the position in the pipeline
# count is the field to sort by, and -1 means to sort in descending order
sort = {
    '$sort': {'count': pymongo.DESCENDING}
}

match_final = {
    "$match": {"count": {"$gt": 100}}
}

cursor = collection.aggregate([match, group, sort, match_final])
for c in cursor:
    print(c)



{'_id': 'American', 'count': 411}
{'_id': 'Chinese', 'count': 323}
{'_id': 'Pizza', 'count': 197}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 187}
{'_id': 'Spanish', 'count': 127}
{'_id': 'Caribbean', 'count': 110}
{'_id': 'Chicken', 'count': 108}


This is a simple query, which shows some of the basic stages of the aggregation pipeline.  It can be improved as follows:

* We can change the name of the `_id` in the output back to `cuisine` using the `$project` stage
* We can change the order of the output to be sorted in alphabetical order as well
* We can limit the results to include results only with a count of 20 or more

Implement those stages in the cell below

In [68]:
# YOUR CODE HERE

project_stage = {
    "$project": {"cuisine": "$_id", "count": 1}
}

sort_stage = {
    "$sort": {"cuisine": pymongo.ASCENDING}
}

match_stage = {
    "$match": {"count": {"$gt":20} }
}

pipeline = [match, group, project_stage, match_stage, sort_stage] #More code here...
cursor = collection.aggregate(pipeline)
for c in cursor:
    print(c)


{'_id': 'African', 'count': 31, 'cuisine': 'African'}
{'_id': 'American', 'count': 411, 'cuisine': 'American'}
{'_id': 'Bakery', 'count': 71, 'cuisine': 'Bakery'}
{'_id': 'Café/Coffee/Tea', 'count': 45, 'cuisine': 'Café/Coffee/Tea'}
{'_id': 'Caribbean', 'count': 110, 'cuisine': 'Caribbean'}
{'_id': 'Chicken', 'count': 108, 'cuisine': 'Chicken'}
{'_id': 'Chinese', 'count': 323, 'cuisine': 'Chinese'}
{'_id': 'Delicatessen', 'count': 26, 'cuisine': 'Delicatessen'}
{'_id': 'Donuts', 'count': 68, 'cuisine': 'Donuts'}
{'_id': 'Hamburgers', 'count': 78, 'cuisine': 'Hamburgers'}
{'_id': 'Ice Cream, Gelato, Yogurt, Ices', 'count': 27, 'cuisine': 'Ice Cream, Gelato, Yogurt, Ices'}
{'_id': 'Italian', 'count': 52, 'cuisine': 'Italian'}
{'_id': 'Juice, Smoothies, Fruit Salads', 'count': 35, 'cuisine': 'Juice, Smoothies, Fruit Salads'}
{'_id': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 'count': 187, 'cuisine': 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)

### Detour: Aggregation with the Zip Code Data Set

[link](https://docs.mongodb.com/manual/tutorial/aggregation-zip-code-data-set/)


#### Import data

In [70]:
%%bash
# Use wget to download the data
wget http://media.mongodb.org/zips.json
# mongoimport is the Mongo command to import data.  
# It specifies the database, collection and format, and import file
# --drop means it's going to drop any collection with the same name which already exists
mongoimport --db mongo_lab --collection zipcodes --drop --file ./zips.json
# Delete the JSON file we just downloaded
rm ./zips.json

--2018-12-07 17:19:06--  http://media.mongodb.org/zips.json
Resolving media.mongodb.org (media.mongodb.org)... 13.32.123.71, 13.32.123.134, 13.32.123.249, ...
Connecting to media.mongodb.org (media.mongodb.org)|13.32.123.71|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3182409 (3.0M) [application/json]
Saving to: ‘zips.json.1’

     0K .......... .......... .......... .......... ..........  1% 2.45M 1s
    50K .......... .......... .......... .......... ..........  3%  784K 3s
   100K .......... .......... .......... .......... ..........  4% 2.71M 2s
   150K .......... .......... .......... .......... ..........  6% 1.33M 2s
   200K .......... .......... .......... .......... ..........  8% 2.02M 2s
   250K .......... .......... .......... .......... ..........  9% 1.67M 2s
   300K .......... .......... .......... .......... .......... 11% 2.29M 2s
   350K .......... .......... .......... .......... .......... 12% 3.99M 2s
   400K .......... .......... .....

In [71]:
db.collection_names()

/home/shaunak/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['restaurants', 'test_collection', 'zipcodes']

Examining structure of a document

In [72]:
collection = db.zipcodes

collection.find_one({})

{'_id': '01005',
 'city': 'BARRE',
 'loc': [-72.108354, 42.409698],
 'pop': 4546,
 'state': 'MA'}

### Return States with Populations above 10 Million

In [74]:
group_stage = {
    "$group": {"_id": "$state", "totalPop": { "$sum": "$pop" }}
}

match_stage = {
    "$match": {"totalPop": {"$gte": 10*1000*1000 }}
}

pipeline = [group_stage, match_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)

{'_id': 'IL', 'totalPop': 11427576}
{'_id': 'OH', 'totalPop': 10846517}
{'_id': 'FL', 'totalPop': 12686644}
{'_id': 'NY', 'totalPop': 17990402}
{'_id': 'PA', 'totalPop': 11881643}
{'_id': 'TX', 'totalPop': 16984601}
{'_id': 'CA', 'totalPop': 29754890}


### Return Average City Population by State

Output:

```

{
  "_id" : "MN",
  "avgCityPop" : 5335
}

```

The first \$group stage groups the documents by the combination of city and state, uses the $sum expression to calculate the population for each combination, and outputs a document for each city and state combination.

In [84]:
group_stage1 = {
    "$group": { "_id": {"state": "$state", "city": "$city"}, "pop": { "$sum": "$pop" }}
}


pipeline = [group_stage1]

for doc in collection.aggregate(pipeline):
    pprint(doc)

{'_id': {'city': 'SITKA', 'state': 'AK'}, 'pop': 8638}
{'_id': {'city': 'CHALKYITSIK', 'state': 'AK'}, 'pop': 99}
{'_id': {'city': 'HYDER', 'state': 'AK'}, 'pop': 116}
{'_id': {'city': 'THORNE BAY', 'state': 'AK'}, 'pop': 744}
{'_id': {'city': 'SKAGWAY', 'state': 'AK'}, 'pop': 692}
{'_id': {'city': 'HOONAH', 'state': 'AK'}, 'pop': 1670}
{'_id': {'city': 'NUIQSUT', 'state': 'AK'}, 'pop': 354}
{'_id': {'city': 'TELLER', 'state': 'AK'}, 'pop': 260}
{'_id': {'city': 'WALES', 'state': 'AK'}, 'pop': 341}
{'_id': {'city': 'WAINWRIGHT', 'state': 'AK'}, 'pop': 492}
{'_id': {'city': 'VENETIE', 'state': 'AK'}, 'pop': 184}
{'_id': {'city': 'TANANA', 'state': 'AK'}, 'pop': 345}
{'_id': {'city': 'SELAWIK', 'state': 'AK'}, 'pop': 0}
{'_id': {'city': 'ANDERSON', 'state': 'AK'}, 'pop': 300}
{'_id': {'city': 'KOTZEBUE', 'state': 'AK'}, 'pop': 3347}
{'_id': {'city': 'POINT HOPE', 'state': 'AK'}, 'pop': 640}
{'_id': {'city': 'GOLOVIN', 'state': 'AK'}, 'pop': 3706}
{'_id': {'city': 'NENANA', 'state': 'AK'}

{'_id': {'city': 'COMPTCHE', 'state': 'CA'}, 'pop': 538}
{'_id': {'city': 'ALBION', 'state': 'CA'}, 'pop': 869}
{'_id': {'city': 'CLOVERDALE', 'state': 'CA'}, 'pop': 7695}
{'_id': {'city': 'CLEARLAKE', 'state': 'CA'}, 'pop': 12157}
{'_id': {'city': 'BRANSCOMB', 'state': 'CA'}, 'pop': 708}
{'_id': {'city': 'SALIDA', 'state': 'CA'}, 'pop': 3255}
{'_id': {'city': 'TRACY', 'state': 'CA'}, 'pop': 47291}
{'_id': {'city': 'SNELLING', 'state': 'CA'}, 'pop': 1335}
{'_id': {'city': 'SOULSBYVILLE', 'state': 'CA'}, 'pop': 1566}
{'_id': {'city': 'NYSSA', 'state': 'OR'}, 'pop': 5288}
{'_id': {'city': 'SONORA', 'state': 'CA'}, 'pop': 23398}
{'_id': {'city': 'RIPON', 'state': 'CA'}, 'pop': 10879}
{'_id': {'city': 'EDGEWOOD', 'state': 'CA'}, 'pop': 5506}
{'_id': {'city': 'PATTERSON', 'state': 'CA'}, 'pop': 13437}
{'_id': {'city': 'PINECREST', 'state': 'CA'}, 'pop': 13}
{'_id': {'city': 'KNIGHTS FERRY', 'state': 'CA'}, 'pop': 20919}
{'_id': {'city': 'LIVINGSTON', 'state': 'CA'}, 'pop': 10994}
{'_id': {'

{'_id': {'city': 'ARRIBA', 'state': 'CO'}, 'pop': 388}
{'_id': {'city': 'BETHUNE', 'state': 'CO'}, 'pop': 543}
{'_id': {'city': 'YUMA', 'state': 'CO'}, 'pop': 3811}
{'_id': {'city': 'SEDGWICK', 'state': 'CO'}, 'pop': 415}
{'_id': {'city': 'STONEHAM', 'state': 'CO'}, 'pop': 141}
{'_id': {'city': 'PADRONI', 'state': 'CO'}, 'pop': 72}
{'_id': {'city': 'PEETZ', 'state': 'CO'}, 'pop': 451}
{'_id': {'city': 'OVID', 'state': 'CO'}, 'pop': 563}
{'_id': {'city': 'OTIS', 'state': 'CO'}, 'pop': 1102}
{'_id': {'city': 'JULESBURG', 'state': 'CO'}, 'pop': 1712}
{'_id': {'city': 'ILIFF', 'state': 'CO'}, 'pop': 778}
{'_id': {'city': 'HALE', 'state': 'CO'}, 'pop': 393}
{'_id': {'city': 'HOLYOKE', 'state': 'CO'}, 'pop': 2642}
{'_id': {'city': 'GROVER', 'state': 'CO'}, 'pop': 492}
{'_id': {'city': 'ECKLEY', 'state': 'CO'}, 'pop': 242}
{'_id': {'city': 'FLEMING', 'state': 'CO'}, 'pop': 853}
{'_id': {'city': 'BRUSH', 'state': 'CO'}, 'pop': 5603}
{'_id': {'city': 'PLATTEVILLE', 'state': 'CO'}, 'pop': 2577}


{'_id': {'city': 'BLUFFTON', 'state': 'AR'}, 'pop': 208}
{'_id': {'city': 'ALIX', 'state': 'AR'}, 'pop': 488}
{'_id': {'city': 'SEAL ROCK', 'state': 'OR'}, 'pop': 1845}
{'_id': {'city': 'RUSSELLVILLE', 'state': 'AR'}, 'pop': 25169}
{'_id': {'city': 'WITTER', 'state': 'AR'}, 'pop': 179}
{'_id': {'city': 'WEST FORK', 'state': 'AR'}, 'pop': 4473}
{'_id': {'city': 'SULPHUR SPRINGS', 'state': 'AR'}, 'pop': 982}
{'_id': {'city': 'SPRINGDALE', 'state': 'AR'}, 'pop': 20104}
{'_id': {'city': 'SILOAM SPRINGS', 'state': 'AR'}, 'pop': 11677}
{'_id': {'city': 'GARFIELD', 'state': 'WA'}, 'pop': 795}
{'_id': {'city': 'SAINT PAUL', 'state': 'AR'}, 'pop': 608}
{'_id': {'city': 'PETTIGREW', 'state': 'AR'}, 'pop': 422}
{'_id': {'city': 'PORT LAVACA', 'state': 'TX'}, 'pop': 15627}
{'_id': {'city': 'MORROW', 'state': 'AR'}, 'pop': 615}
{'_id': {'city': 'HUNTSVILLE', 'state': 'AR'}, 'pop': 7021}
{'_id': {'city': 'PEA RIDGE', 'state': 'AR'}, 'pop': 3584}
{'_id': {'city': 'CUSICK', 'state': 'WA'}, 'pop': 439}

{'_id': {'city': 'MORGANVILLE', 'state': 'KS'}, 'pop': 847}
{'_id': {'city': 'MARQUETTE', 'state': 'KS'}, 'pop': 760}
{'_id': {'city': 'MINNEAPOLIS', 'state': 'KS'}, 'pop': 2985}
{'_id': {'city': 'LORRAINE', 'state': 'KS'}, 'pop': 302}
{'_id': {'city': 'LITTLE RIVER', 'state': 'KS'}, 'pop': 841}
{'_id': {'city': 'KANOPOLIS', 'state': 'KS'}, 'pop': 641}
{'_id': {'city': 'RACKERBY', 'state': 'CA'}, 'pop': 260}
{'_id': {'city': 'WESTFALL', 'state': 'KS'}, 'pop': 1986}
{'_id': {'city': 'HUNTER', 'state': 'KS'}, 'pop': 181}
{'_id': {'city': 'GENESEO', 'state': 'KS'}, 'pop': 569}
{'_id': {'city': 'GALVA', 'state': 'KS'}, 'pop': 1151}
{'_id': {'city': 'HOLYROOD', 'state': 'KS'}, 'pop': 678}
{'_id': {'city': 'CHAPMAN', 'state': 'KS'}, 'pop': 1968}
{'_id': {'city': 'EVANS', 'state': 'WA'}, 'pop': 350}
{'_id': {'city': 'HAMMON', 'state': 'OK'}, 'pop': 873}
{'_id': {'city': 'ENTERPRISE', 'state': 'KS'}, 'pop': 1286}
{'_id': {'city': 'APPLEGATE', 'state': 'OR'}, 'pop': 5723}
{'_id': {'city': 'CAWK

...

{'_id': {'city': 'SAPPHIRE', 'state': 'NC'}, 'pop': 62}
{'_id': {'city': 'REGAN', 'state': 'ND'}, 'pop': 274}
{'_id': {'city': 'BEACONSFIELD', 'state': 'IA'}, 'pop': 195}
{'_id': {'city': 'OAKLYN', 'state': 'NJ'}, 'pop': 14161}
{'_id': {'city': 'SOUTHAMPTON', 'state': 'NJ'}, 'pop': 25480}
{'_id': {'city': 'HARTVILLE', 'state': 'OH'}, 'pop': 8318}
{'_id': {'city': 'HALLS', 'state': 'TN'}, 'pop': 3755}
{'_id': {'city': 'MILAN', 'state': 'TN'}, 'pop': 11377}
{'_id': {'city': 'BELLEVILLE', 'state': 'AR'}, 'pop': 988}
{'_id': {'city': 'WEST CONSHOHOCKE', 'state': 'PA'}, 'pop': 15924}
{'_id': {'city': 'PAWLING', 'state': 'NY'}, 'pop': 4511}
{'_id': {'city': 'GRANITE', 'state': 'CO'}, 'pop': 79}
{'_id': {'city': 'CRAWFORD', 'state': 'OK'}, 'pop': 178}
{'_id': {'city': 'WOODSTOWN', 'state': 'NJ'}, 'pop': 8013}
{'_id': {'city': 'LEBANON', 'state': 'IN'}, 'pop': 17322}
{'_id': {'city': 'PRATTVILLE', 'state': 'AL'}, 'pop': 27595}
{'_id': {'city': 'COLEVILLE', 'state': 'CA'}, 'pop': 1370}
{'_id': 

{'_id': {'city': 'PRINCETON', 'state': 'MO'}, 'pop': 2467}
{'_id': {'city': 'CRESSKILL', 'state': 'NJ'}, 'pop': 7633}
{'_id': {'city': 'CAVENDISH', 'state': 'VT'}, 'pop': 258}
{'_id': {'city': 'WOODSTOCK', 'state': 'VT'}, 'pop': 3742}
{'_id': {'city': 'PATTENBURG', 'state': 'NJ'}, 'pop': 2176}
{'_id': {'city': 'SAINT CHARLES', 'state': 'ID'}, 'pop': 199}
{'_id': {'city': 'MANSFIELD CENTER', 'state': 'CT'}, 'pop': 4306}
{'_id': {'city': 'ELKINS PARK', 'state': 'PA'}, 'pop': 12813}
{'_id': {'city': 'MORAVIA', 'state': 'NY'}, 'pop': 6267}
{'_id': {'city': 'BELLOWS FALLS', 'state': 'VT'}, 'pop': 5433}
{'_id': {'city': 'STERLING CITY', 'state': 'TX'}, 'pop': 1438}
{'_id': {'city': 'ULER', 'state': 'WV'}, 'pop': 724}
{'_id': {'city': 'CAMBRIDGEPORT', 'state': 'VT'}, 'pop': 118}
{'_id': {'city': 'CEDAR', 'state': 'MI'}, 'pop': 2853}
{'_id': {'city': 'AFTON', 'state': 'TX'}, 'pop': 132}
{'_id': {'city': 'BURLINGTON', 'state': 'VT'}, 'pop': 39127}
{'_id': {'city': 'ROCHESTER HILLS', 'state': 'M

{'_id': {'city': 'MIRROR LAKE', 'state': 'NH'}, 'pop': 696}
{'_id': {'city': 'FOMBELL', 'state': 'PA'}, 'pop': 2611}
{'_id': {'city': 'MOROCCO', 'state': 'IN'}, 'pop': 2461}
{'_id': {'city': 'POWELL', 'state': 'TN'}, 'pop': 14646}
{'_id': {'city': 'TRIPLETT', 'state': 'MO'}, 'pop': 192}
{'_id': {'city': 'HAWKINS', 'state': 'WI'}, 'pop': 747}
{'_id': {'city': 'NEW PROVIDENCE', 'state': 'NJ'}, 'pop': 11838}
{'_id': {'city': 'MADISON', 'state': 'NH'}, 'pop': 1669}
{'_id': {'city': 'JACKSON', 'state': 'NH'}, 'pop': 689}
{'_id': {'city': 'WOOD LAKE', 'state': 'NE'}, 'pop': 200}
{'_id': {'city': 'SELBYVILLE', 'state': 'WV'}, 'pop': 40}
{'_id': {'city': 'HENDERSON', 'state': 'TX'}, 'pop': 19836}
{'_id': {'city': 'BURKHART', 'state': 'KY'}, 'pop': 233}
{'_id': {'city': 'SOUTH GLASTONBUR', 'state': 'CT'}, 'pop': 3614}
{'_id': {'city': 'HEALDTON', 'state': 'OK'}, 'pop': 3114}
{'_id': {'city': 'HAMPTON FALLS', 'state': 'NH'}, 'pop': 1503}
{'_id': {'city': 'HICKSVILLE', 'state': 'NY'}, 'pop': 3830

{'_id': {'city': 'DODDRIDGE', 'state': 'AR'}, 'pop': 1435}
{'_id': {'city': 'FRENCH CAMP', 'state': 'MS'}, 'pop': 1426}
{'_id': {'city': 'LAUREL SPRINGS', 'state': 'NC'}, 'pop': 1837}
{'_id': {'city': 'FRANKLIN', 'state': 'MA'}, 'pop': 22128}
{'_id': {'city': 'WESTFORD', 'state': 'VT'}, 'pop': 1102}
{'_id': {'city': 'MARSHFIELD', 'state': 'MA'}, 'pop': 21782}
{'_id': {'city': 'HUGHESVILLE', 'state': 'PA'}, 'pop': 6489}
{'_id': {'city': 'CHIPPEWA LAKE', 'state': 'OH'}, 'pop': 2714}
{'_id': {'city': 'MARVELL', 'state': 'AR'}, 'pop': 2733}
{'_id': {'city': 'MINDEN', 'state': 'NE'}, 'pop': 4365}
{'_id': {'city': 'LICKINGVILLE', 'state': 'PA'}, 'pop': 22}
{'_id': {'city': 'GRANDVIEW', 'state': 'WA'}, 'pop': 10558}
{'_id': {'city': 'WESTERLY', 'state': 'RI'}, 'pop': 20290}
{'_id': {'city': 'EAST GREENBUSH', 'state': 'NY'}, 'pop': 7282}
{'_id': {'city': 'EAST WALPOLE', 'state': 'MA'}, 'pop': 3844}
{'_id': {'city': 'FLINTVILLE', 'state': 'TN'}, 'pop': 6020}
{'_id': {'city': 'BRITTON', 'state':

{'_id': {'city': 'WARE', 'state': 'MA'}, 'pop': 9808}
{'_id': {'city': 'SOUTH KENT', 'state': 'CT'}, 'pop': 719}
{'_id': {'city': 'HINES', 'state': 'MN'}, 'pop': 1083}
{'_id': {'city': 'MOUNT LAUREL', 'state': 'NJ'}, 'pop': 29805}
{'_id': {'city': 'YALE', 'state': 'VA'}, 'pop': 626}
{'_id': {'city': 'WENDELL', 'state': 'MA'}, 'pop': 393}
{'_id': {'city': 'ATTLEBORO', 'state': 'MA'}, 'pop': 38528}
{'_id': {'city': 'LEBANON CHURCH', 'state': 'VA'}, 'pop': 1075}
{'_id': {'city': 'NEW WESTON', 'state': 'OH'}, 'pop': 1266}
{'_id': {'city': 'SOUTH CHINA', 'state': 'ME'}, 'pop': 182}
{'_id': {'city': 'NORTH NEW PORTLA', 'state': 'ME'}, 'pop': 537}
{'_id': {'city': 'HAYDENVILLE', 'state': 'MA'}, 'pop': 1387}
{'_id': {'city': 'WELLINGTON', 'state': 'AL'}, 'pop': 2137}
{'_id': {'city': 'CLYMER', 'state': 'NY'}, 'pop': 2810}
{'_id': {'city': 'RINCON', 'state': 'GA'}, 'pop': 14502}
{'_id': {'city': 'LEEDS', 'state': 'MA'}, 'pop': 1350}
{'_id': {'city': 'LONGMEADOW', 'state': 'MA'}, 'pop': 15688}
{

A second \$group stage groups the documents in the pipeline by the _id.state field (i.e. the state field inside the _id document), uses the \$avg expression to calculate the average city population (avgCityPop) for each state, and outputs a document for each state.

In [90]:
group_stage2 = {
    "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" }}
}

sort_stage = {
    "$sort": {'_id': pymongo.DESCENDING}
}


pipeline = [group_stage1, group_stage2, sort_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)


{'_id': 'WY', 'avgCityPop': 3384.5373134328356}
{'_id': 'WV', 'avgCityPop': 2771.4775888717154}
{'_id': 'WI', 'avgCityPop': 7323.00748502994}
{'_id': 'WA', 'avgCityPop': 12258.670025188916}
{'_id': 'VT', 'avgCityPop': 2315.8765432098767}
{'_id': 'VA', 'avgCityPop': 8526.177931034483}
{'_id': 'UT', 'avgCityPop': 9518.508287292818}
{'_id': 'TX', 'avgCityPop': 13775.02108678021}
{'_id': 'TN', 'avgCityPop': 9656.350495049504}
{'_id': 'SD', 'avgCityPop': 1839.6746031746031}
{'_id': 'SC', 'avgCityPop': 11139.626198083068}
{'_id': 'RI', 'avgCityPop': 19292.653846153848}
{'_id': 'PA', 'avgCityPop': 8679.067202337472}
{'_id': 'OR', 'avgCityPop': 8262.561046511628}
{'_id': 'OK', 'avgCityPop': 6155.743639921722}
{'_id': 'OH', 'avgCityPop': 12700.839578454332}
{'_id': 'NY', 'avgCityPop': 13131.680291970803}
{'_id': 'NV', 'avgCityPop': 18209.590909090908}
{'_id': 'NM', 'avgCityPop': 5872.360465116279}
{'_id': 'NJ', 'avgCityPop': 15775.89387755102}
{'_id': 'NH', 'avgCityPop': 5232.320754716981}
{'_i

### Return Largest and Smallest Cities by State


In [106]:
group_stage1 = {
    "$group": { "_id": {"state": "$state", "city": "$city"}, "pop": { "$sum": "$pop" }}
}

sort_stage = {
    "$sort": {"pop": 1}
}

group_stage2 = {
    "$group": {"_id": "$_id.state", 
                   "biggestCity": {"$last": "$_id.city"},
                   "biggestPop": {"$last": "$pop"},
                   "smallestCity": {"$first": "$_id.city"},
                   "smallestPop": {"$first": "$pop"},
              
              }
}

project_stage = {
    "$project": {
        "_id": 0,
        "state:": "$_id",
        "biggestCityData": {"name": "$biggestCity", "pop": "$biggestPop"},
        "smallestCityData": {"name": "$smallestCity", "pop": "$smallestPop"}
    }
}


pipeline = [group_stage1, sort_stage, group_stage2, project_stage]

for doc in collection.aggregate(pipeline):
    pprint(doc)



{'biggestCityData': {'name': 'NEWARK', 'pop': 111674},
 'smallestCityData': {'name': 'BETHEL', 'pop': 108},
 'state:': 'DE'}
{'biggestCityData': {'name': 'SAINT LOUIS', 'pop': 397802},
 'smallestCityData': {'name': 'BENDAVIS', 'pop': 44},
 'state:': 'MO'}
{'biggestCityData': {'name': 'CHICAGO', 'pop': 2452177},
 'smallestCityData': {'name': 'ANCONA', 'pop': 38},
 'state:': 'IL'}
{'biggestCityData': {'name': 'CLEVELAND', 'pop': 536759},
 'smallestCityData': {'name': 'ISLE SAINT GEORG', 'pop': 38},
 'state:': 'OH'}
{'biggestCityData': {'name': 'MANCHESTER', 'pop': 106452},
 'smallestCityData': {'name': 'WEST NOTTINGHAM', 'pop': 27},
 'state:': 'NH'}
{'biggestCityData': {'name': 'WASHINGTON', 'pop': 606879},
 'smallestCityData': {'name': 'PENTAGON', 'pop': 21},
 'state:': 'DC'}
{'biggestCityData': {'name': 'GRAND FORKS', 'pop': 59527},
 'smallestCityData': {'name': 'TROTTERS', 'pop': 12},
 'state:': 'ND'}
{'biggestCityData': {'name': 'BALTIMORE', 'pop': 733081},
 'smallestCityData': {'nam

In [ ]:
# YOUR CODE HERE FOR sort

pipeline.append(sort)
cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


In [ ]:
# YOUR CODE HERE FOR `count` > 20

cursor = collection.aggregate(pipeline)
for c in cursor:
    pprint(c)


## Challenge

How would you work out the percentage of each type of cuisine out of all selected restaurants?

In [ ]:
# YOUR CODE HERE...

# Where next?

* Dealing with array data
* Update, delete, and drop
* Setting up authentication
* Sharding databases